# Árbol de decisión para clasificación

In [ ]:
import pandas as pd

penguins = pd.read_csv("../../data/penguins/penguins_classification.csv")
culmen_columns = ["Culmen Length (mm)", "Culmen Depth (mm)"]
target_column = "Species"

In [ ]:
# dividir los datos

from sklearn.model_selection import train_test_split

data, target = penguins[culmen_columns], penguins[target_column]
data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=0)

**Clasificador lineal**

In [ ]:
# Para un clasificador lineal, obtendremos los siguientes límites de decisión.
# Estas líneas de límites indican dónde cambia el modelo su predicción de una clase a otra.

from sklearn.linear_model import LogisticRegression

linear_model = LogisticRegression()
linear_model.fit(data_train, target_train)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.inspection import DecisionBoundaryDisplay

# Cree una paleta para ser utilizada en el diagrama de dispersión
palette = ["tab:red", "tab:blue", "black"]

DecisionBoundaryDisplay.from_estimator(
    linear_model, data_train, response_method="predict", cmap="RdBu", alpha=0.5
)
sns.scatterplot(data=penguins, x=culmen_columns[0], y=culmen_columns[1],
                hue=target_column, palette=palette)

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
_ = plt.title("Límite de decisión utilizando una regresión logística")

In [ ]:
# parece que el modelo lineal sería un buen candidato para el problema, ya que da una buena precisión.

linear_model.fit(data_train, target_train)
test_score = linear_model.score(data_test, target_test)
print(f"Precisión de la redacción logística: {test_score:.2f}")

**Árboles de decisión**

In [ ]:
# Los árboles de decisión dividirán el espacio considerando una sola característica a la vez. 
# Ilustramos este comportamiento al hacer que un árbol de decisión haga una sola división al dividir el espacio de características.

from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=1)
tree.fit(data_train, target_train)

In [ ]:
DecisionBoundaryDisplay.from_estimator(
    tree, data_train, response_method="predict", cmap="RdBu", alpha=0.5
)
sns.scatterplot(data=penguins, x=culmen_columns[0], y=culmen_columns[1],
                hue=target_column, palette=palette)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
_ = plt.title("Límite de decisión utilizando un árbol de decisión")

Las particiones encontradas por el algoritmo separan los datos a lo largo del eje “Culmen Depth”

In [ ]:
# Podemos ver más en detalle la estructura del árbol.

from sklearn.tree import plot_tree

_, ax = plt.subplots(figsize=(8, 6))
_ = plot_tree(tree, feature_names=culmen_columns,
              class_names=tree.classes_, impurity=False, ax=ax)

In [ ]:
# Veamos cómo funcionaría nuestro árbol como predictor. 
# Comencemos con un caso en el que la profundidad de los culmen sea inferior al umbral.

sample_1 = pd.DataFrame(
    {"Culmen Length (mm)": [0], "Culmen Depth (mm)": [15]}
)
tree.predict(sample_1)

In [ ]:
# La clase predicha es Gentoo. 
# Podemos verificar qué sucede si pasamos una profundidad de culmen superior al umbral.

sample_2 = pd.DataFrame(
    {"Culmen Length (mm)": [0], "Culmen Depth (mm)": [17]}
)
tree.predict(sample_2)

En este caso, el árbol predice la especie de Adelie.

-> podemos concluir que un clasificador de árbol de decisión predecirá la clase más representada dentro de una partición.

In [40]:
# Durante el entrenamiento, tenemos un recuento de muestras en cada partición, 
# también podemos calcular la probabilidad de pertenecer a una clase específica dentro de esta partición.

y_pred_proba = tree.predict_proba(sample_2)
y_proba_class_0 = pd.Series(y_pred_proba[0], index=tree.classes_)

In [ ]:
y_proba_class_0.plot.bar()
plt.ylabel("Probabilidad")
_ = plt.title("Probabilidad de pertenecer a una clase de pingüino")

In [ ]:
# podemos calcular las diferentes probabilidades manualmente directamente de la estructura del árbol.

adelie_proba = 103 / 161
chinstrap_proba = 52 / 161
gentoo_proba = 6 / 161
print(
    f"Probabilidades para las diferentes clases:\n"
    f"Adelie: {adelie_proba:.3f}\n"
    f"Chinstrap: {chinstrap_proba:.3f}\n"
    f"Gentoo: {gentoo_proba:.3f}\n"
)

In [ ]:
# También es importante tener en cuenta que la longitud de culmen se ha ignorado de momento: 
#  sea cual sea el valor dado, no se usará durante la predicción.

sample_3 = pd.DataFrame(
    {"Culmen Length (mm)": [10_000], "Culmen Depth (mm)": [17]}
)
tree.predict_proba(sample_3)

In [ ]:
# La división encontrada con una profundidad máxima de 1 no es lo suficientemente potente como para separar las tres especies 
# y la precisión del modelo es baja en comparación con el modelo lineal.

tree.fit(data_train, target_train)
test_score = tree.score(data_test, target_test)
print(f"Precisión de la decisión: {test_score:.2f}")

## Ejercicio:
- Aumenta progresivamente la profundidad y observa el comportamiento del modelo